# Part 0
## Intro
well I want to try selenium
so
here we go

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import pandas as pd
import time

# Part 1
## Collect links

In [2]:
city_name = "Saint Petersburg"
delay = 5 # seconds

links = {}

In [3]:
driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")
driver.get("https://www.tripadvisor.com/Search?q=parks&searchSessionId=CF4A26D0C451E0783853C96C46AC8B131637532036921ssid&sid=72ED914F7F2B4263B4C7FB9FF61A5A671637532046511&geo=186338&blockRedirect=true&ssrc=a")
javaScript = "PagerNextClick();"

inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys('Moscow')
time.sleep(2)
clearElement = driver.find_element_by_id("CLEAR_WHERE")
clearElement.click()
time.sleep(2)
inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys(city_name)
time.sleep(2)
inputElement.send_keys(Keys.ENTER)
time.sleep(3)
parksElement = driver.find_elements_by_xpath("//*[contains(text(), 'Parks in')]")
parksElement[1].click()

C:\Users\wirze\AppData\Local\Temp/ipykernel_1060/463546191.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")
C:\Users\wirze\AppData\Local\Temp/ipykernel_1060/463546191.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
C:\Users\wirze\AppData\Local\Temp/ipykernel_1060/463546191.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  clearElement = driver.find_element_by_id("CLEAR_WHERE")
C:\Users\wirze\AppData\Local\Temp/ipykernel_1060/463546191.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")


KeyboardInterrupt: 

In [330]:
for i in range(42):
    html = driver.page_source
    soup = BeautifulSoup(html)

    results_names = soup.find_all("div", class_="csemS")
    results_links = soup.find_all("div", class_="fVbwn cdAAV cagLQ eZTON dofsx")
    for result_link in results_links:
        if result_link.find("a") is None:
            results_links.remove(result_link)

    for result_link, result_name in zip(results_links, results_names):
        if result_link.find("a") is not None:
            links[result_name.text] = result_link.find("a")['href']
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'csemS')))
        NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")
        time.sleep(5)

    if len(NextPageElement) == 0:
        break
    NextPageElement[0].click()
    time.sleep(5)

C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/954684860.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')


Page is ready!
Page is ready!
Page is ready!


In [331]:
print(list(links)[1])
print(list(links)[-1])

3. Zelenogorsk Recreation Park
75. Nizhniy Park


In [332]:
df_links_all =  pd.read_csv('all_parks_links.csv')

df_links = pd.DataFrame.from_dict(links,  orient='index', columns=["link"]).reset_index()
df_links['city'] = city_name
df_links = df_links[['city', 'index', 'link']]

df_links_all2 = pd.concat([df_links_all, df_links], ignore_index=True)
df_links_all2 = df_links_all2[['city', 'index', 'link']]
df_links_all2.to_csv("all_parks_links.csv", index=False)

In [333]:
df_links_all =  pd.read_csv('all_parks_links.csv')
df_links_all

,city,index,link
0,test_city,test_park,test_link
1,Moscow,1. Victory Park,/Attraction_Review-g298484-d300387-Reviews-Vic...
2,Moscow,2. Gorkiy Central Park of Culture and Recreation,/Attraction_Review-g298484-d300383-Reviews-Gor...
3,Moscow,3. Sokolniki Park,/Attraction_Review-g298484-d565309-Reviews-Sok...
4,Moscow,4. Neskuchny sad,/Attraction_Review-g298484-d2342077-Reviews-Ne...
...,...,...,...
1421,Saint Petersburg,71. Metallist Skver,/Attraction_Review-g2394587-d21039548-Reviews-...
1422,Saint Petersburg,72. Nizhniy Park,/Attraction_Review-g811323-d21000965-Reviews-N...
1423,Saint Petersburg,73. Полюстровский парк,/Attraction_Review-g298507-d23539638-Reviews-S...
1424,Saint Petersburg,74. English Park,/Attraction_Review-g1207894-d23606038-Reviews-...


In [334]:
driver.quit()

# Part 2
## Reviews

In [335]:
df_links_all.city.unique()

array(['test_city', 'Moscow', 'London', 'Chicago', 'Paris', 'New York',
       'San Francisco', 'Bangkok', 'Singapore', 'Istanbul', 'Tokyo',
       'Saint Petersburg'], dtype=object)

In [336]:
city_name = "Saint Petersburg"

In [337]:
df_links_city = df_links_all[df_links_all['city'] == city_name]
df_links_city

,city,index,link
1352,Saint Petersburg,1. Catherine Palace and Park,/Attraction_Review-g811323-d301025-Reviews-Cat...
1353,Saint Petersburg,3. Zelenogorsk Recreation Park,/Attraction_Review-g811323-d301026-Reviews-Ale...
1354,Saint Petersburg,4. Dubki Park of Culture and Leisure,/Attraction_Review-g2345785-d6502907-Reviews-Z...
1355,Saint Petersburg,5. Park And Gardens of Peterhof,/Attraction_Review-g2345793-d6502231-Reviews-D...
1356,Saint Petersburg,6. Peterhof State Museum Preserve,/Attraction_Review-g1207894-d10533361-Reviews-...
...,...,...,...
1421,Saint Petersburg,71. Metallist Skver,/Attraction_Review-g2394587-d21039548-Reviews-...
1422,Saint Petersburg,72. Nizhniy Park,/Attraction_Review-g811323-d21000965-Reviews-N...
1423,Saint Petersburg,73. Полюстровский парк,/Attraction_Review-g298507-d23539638-Reviews-S...
1424,Saint Petersburg,74. English Park,/Attraction_Review-g1207894-d23606038-Reviews-...


In [338]:
reviews_df = pd.DataFrame(data={'id':[0], 'text':['park4'], 'link':['www']})
reviews_df.to_csv(city_name.lower()+'_parks_reviews.csv', index=False)
reviews_df

,id,text,link
0,0,park4,www


In [341]:
df_links_city.link[0:4].tolist()

['/Attraction_Review-g811323-d301025-Reviews-Catherine_Palace_and_Park-Pushkin_Pushkinsky_District_St_Petersburg_Northwestern_D.html',
 '/Attraction_Review-g811323-d301026-Reviews-Alexander_Park-Pushkin_Pushkinsky_District_St_Petersburg_Northwestern_District.html',
 '/Attraction_Review-g2345785-d6502907-Reviews-Zelenogorsk_Recreation_Park-Zelenogorsk_Kurortny_District_St_Petersburg_Northwes.html',
 '/Attraction_Review-g2345793-d6502231-Reviews-Dubki_Park_of_Culture_and_Leisure-Sestroretsk_Kurortny_District_St_Petersburg_No.html',
 '/Attraction_Review-g1207894-d10533361-Reviews-Park_And_Gardens_of_Peterhof-Peterhof_Petrodvortsovy_District_St_Petersburg_Nor.html']

In [342]:
driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")

for link in df_links_city.link:
    link_to_park = "https://www.tripadvisor.com"+link
    if link not in df_links_city.link[0:4].tolist():
        print(link_to_park)
        i = 1
        reviews = {}
        driver.get(link_to_park)
        time.sleep(5)
        for j in range(50):

            html = driver.page_source
            soup = BeautifulSoup(html)

            results_bodies = soup.find_all("div", class_="pIRBV _T KRIav")
            for body in results_bodies:
                if body.find("span", class_="cSoNT") is not None:
                    reviews[i] = body.find("span", class_="cSoNT").text
                    i += 1
            try:
                myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'KRIav')))
                time.sleep(1)
                NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')
                print ("Page is ready!")
            except TimeoutException:
                print ("Loading took too much time!")
                driver.refresh()
                time.sleep(3)
            if len(NextPageElement) == 0:
                print(i)
                break
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[aria-label="Next page"]'))).click()
            time.sleep(2)

        time.sleep(5)
        df_reviews_park = pd.DataFrame.from_dict(reviews,  orient='index', columns=["text"]).reset_index()
        df_reviews_park = df_reviews_park.rename(columns={"index": "id"})
        df_reviews_park['link'] = link_to_park
        df_reviews_park

        reviews_df = pd.read_csv(city_name.lower()+"_parks_reviews.csv")
        reviews_df

        reviews_df2 = pd.concat([reviews_df, df_reviews_park], ignore_index=True)
        reviews_df2 = reviews_df2[['id', 'text', 'link']]
        reviews_df2.to_csv(city_name.lower()+"_parks_reviews.csv", index=False)

C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/3790441302.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")


https://www.tripadvisor.com/Attraction_Review-g1207894-d10533361-Reviews-Park_And_Gardens_of_Peterhof-Peterhof_Petrodvortsovy_District_St_Petersburg_Nor.html


C:\Users\wirze\AppData\Local\Temp/ipykernel_17200/3790441302.py:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')


Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
https://www.tripadvisor.com/Attraction_Review-g1207894-d2615834-Reviews-Peterhof_State_Museum_Preserve-Peterhof_Petrodvortsovy_District_St_Petersburg_No.html
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Pa

In [344]:
reviews_df

,id,text,link
0,0,park4,www
1,1,This is a smaller scale beautiful palace with ...,https://www.tripadvisor.com/Attraction_Review-...
2,2,Absolutely gorgeous. When you see this palace ...,https://www.tripadvisor.com/Attraction_Review-...
3,3,Very pretty palace with nice reception rooms. ...,https://www.tripadvisor.com/Attraction_Review-...
4,4,Visited here in October 2018 on a group tour w...,https://www.tripadvisor.com/Attraction_Review-...
...,...,...,...
1945,1,На углу Петровской улицы и улицы Аммермана был...,https://www.tripadvisor.com/Attraction_Review-...
1946,1,"Цена за посещение завышена, да еще и закрыты в...",https://www.tripadvisor.com/Attraction_Review-...
1947,2,"Вход бесплатный, есть маленький пляж и водоём....",https://www.tripadvisor.com/Attraction_Review-...
1948,1,В историческом Полюстровском парке мне понрави...,https://www.tripadvisor.com/Attraction_Review-...


In [244]:
reviews_df = pd.read_csv("moscow_parks_reviews.csv")
reviews_df

# reviews_df2 = pd.concat([reviews_df, df_reviews_park], ignore_index=True)
# reviews_df2 = reviews_df2[['id', 'text', 'link']]
# reviews_df2.to_csv("moscow_parks_reviews.csv", index=False)